#### General parameters

In [1]:
# The name of the batch or the name of the forlder were programs are
batchName = 'batch_demo'

# The ID of the first progam to be generated, serves as a seed for the random generator
first_program_id = 2020
# Number of functions to be generated
number_of_functions = 15
# Number of different schedules to generate for each program
nb_schedule_per_function = 32

# Path to the directory containing the programs
data_path = './data/'+batchName+'/programs/'
# Number of nodes in the cluster, Each node will do the job on a portion of the programs
nb_nodes = 10
# Path of the temporary files folder
tmp_files_dir = './time_measurement/'
# Path to the script that compile generators
generator_script = './compile_tiramisu_code.py'
# Path of to the script that compiles wrappers
wrappers_script = './compile_tiramisu_wrappers.py'
# Path to the script that execute the compiled wrappers
execute_script = './execute_programs.py'
# Path to where to store the logs of the jobs
log_path = tmp_files_dir + "log_"+batchName+"/"
# Path where to strore the resulting dataset file 
result_dataset_file = tmp_files_dir+'/processed_datasets/dataset_'+batchName+'.pkl'

#please edit the paths accordingly in compile_tiramisu_code.py, compile_tiramisu_wrappers.py, execute programs.py

In [2]:
import pickle
from pathlib import Path
import numpy as np
import pandas as pd
import dill
from os import listdir
import json 
from tqdm import tqdm
import re
import subprocess
import random 
import time 
from multiprocessing import Pool
from shutil import rmtree
import sys
sys.path.append("TiramisuCodeGenerator")
from TiramisuCodeGenerator import code_generator
from data_gen_utils import cluster_utilities, annotation_utilities, dataset_utilities


## 1. Generate synthetic Tiramisu programs

In [3]:
# Generate programs using the program generator
code_generator.generate_programs(seed=first_program_id, number_of_functions=number_of_functions, nb_schedule_per_function=nb_schedule_per_function, batchName=batchName, output_dir = data_path)

cluster_utils = cluster_utilities(data_path, generator_script, wrappers_script, execute_script, log_path, batchName, nb_nodes, tmp_files_dir)

# Create the list of programs that are in the given path
cluster_utils.generate_prog_list()

Generating  15  programs to  ./data/batch_demo/programs/


100%|██████████| 15/15 [00:02<00:00,  6.67it/s]


Done
Total number of schedules generated 396


## 2. Generators' Compilation and Execution

In [4]:
## Generate job files for compiling Tiramisu code
cluster_utils.generate_compile_jobs()

#### Submit generators' compilation and execution jobs

In [5]:
cluster_utils.submit_compile_jobs()

Submitted batch job 15601
Submitted batch job 15602
Submitted batch job 15603
Submitted batch job 15604
Submitted batch job 15605
Submitted batch job 15606
Submitted batch job 15607
Submitted batch job 15608
Submitted batch job 15609
Submitted batch job 15610



In [7]:
# print(cluster_utils.compile_jobs_ids)

In [9]:
cluster_utils.check_compile_progress()

             JOBID PARTITION            NAME     USER ST       TIME  NODES NODELIST(REASON)
             15610  lanka-v3 comp2_batch_dem mmerouan  R       1:08      1 lanka38
             15601  lanka-v3 comp0_batch_dem mmerouan  R       1:09      1 lanka25
             15602  lanka-v3 comp3_batch_dem mmerouan  R       1:09      1 lanka32
             15603  lanka-v3 comp4_batch_dem mmerouan  R       1:09      1 lanka27
             15604  lanka-v3 comp5_batch_dem mmerouan  R       1:09      1 lanka28
             15605  lanka-v3 comp6_batch_dem mmerouan  R       1:09      1 lanka29
             15606  lanka-v3 comp7_batch_dem mmerouan  R       1:09      1 lanka34
             15607  lanka-v3 comp8_batch_dem mmerouan  R       1:09      1 lanka35
             15608  lanka-v3 comp9_batch_dem mmerouan  R       1:09      1 lanka36
             15609  lanka-v3 comp1_batch_dem mmerouan  R       1:09      1 lanka37

==> ./time_measurement/log_batch_demo/log_comp_0_0_39 <==
  }
}


==> ./time_

## 3. Wrappers' Compilation

In [10]:
# Generate compile_wrappers and execute job files
cluster_utils.generate_wrapper_jobs()

#### Submit wrapper's compilation jobs

In [11]:
cluster_utils.submit_wrapper_compilation_jobs()

Submitted batch job 15611
Submitted batch job 15612
Submitted batch job 15613
Submitted batch job 15614
Submitted batch job 15615
Submitted batch job 15616
Submitted batch job 15617
Submitted batch job 15618
Submitted batch job 15619
Submitted batch job 15620



In [8]:
# print(cluster_utils.wrap_jobs_ids)

In [13]:
cluster_utils.check_wrapper_compilation_progress()

             JOBID PARTITION            NAME     USER ST       TIME  NODES NODELIST(REASON)
             15611  lanka-v3 wrap0_batch_dem mmerouan  R       0:17      1 lanka25
             15612  lanka-v3 wrap3_batch_dem mmerouan  R       0:17      1 lanka32
             15613  lanka-v3 wrap4_batch_dem mmerouan  R       0:17      1 lanka27
             15614  lanka-v3 wrap5_batch_dem mmerouan  R       0:17      1 lanka28
             15615  lanka-v3 wrap6_batch_dem mmerouan  R       0:17      1 lanka29
             15616  lanka-v3 wrap7_batch_dem mmerouan  R       0:17      1 lanka34
             15617  lanka-v3 wrap8_batch_dem mmerouan  R       0:17      1 lanka35
             15618  lanka-v3 wrap9_batch_dem mmerouan  R       0:17      1 lanka36
             15619  lanka-v3 wrap1_batch_dem mmerouan  R       0:17      1 lanka37
             15620  lanka-v3 wrap2_batch_dem mmerouan  R       0:17      1 lanka38

==> ./time_measurement/log_batch_demo/log_wrap_0_0_39 <==

==> ./time_measure

## 4. Execute Programs

In [15]:
cluster_utils.generate_execution_slurm_script()

#### Submit execution jobs

In [21]:
#submit exec jobs
cluster_utils.submit_execution_jobs()

Submitted batch job 15631
Submitted batch job 15632
Submitted batch job 15633
Submitted batch job 15634
Submitted batch job 15635
Submitted batch job 15636
Submitted batch job 15637
Submitted batch job 15638
Submitted batch job 15639
Submitted batch job 15640



In [22]:
print(cluster_utils.exec_jobs_ids)

15631,15632,15633,15634,15635,15636,15637,15638,15639,15640


In [29]:
cluster_utils.check_execution_progress()

             JOBID PARTITION            NAME     USER ST       TIME  NODES NODELIST(REASON)
             15631  lanka-v3 exec0_batch_dem mmerouan  R       2:03      1 lanka25
             15632  lanka-v3 exec3_batch_dem mmerouan  R       2:03      1 lanka32
             15634  lanka-v3 exec5_batch_dem mmerouan  R       2:03      1 lanka28
             15635  lanka-v3 exec6_batch_dem mmerouan  R       2:03      1 lanka29
             15636  lanka-v3 exec7_batch_dem mmerouan  R       2:03      1 lanka34
             15637  lanka-v3 exec8_batch_dem mmerouan  R       2:03      1 lanka35
             15638  lanka-v3 exec9_batch_dem mmerouan  R       2:03      1 lanka36
             15639  lanka-v3 exec1_batch_dem mmerouan  R       2:03      1 lanka37
             15640  lanka-v3 exec2_batch_dem mmerouan  R       2:03      1 lanka38

==> ./time_measurement/log_batch_demo/log_exec_0_0_39 <==
2020-10-28 12:22:46.259044-> 0/39
2020-10-28 12:23:36.312226-> 20/39

==> ./time_measurement/log_batch

## 5. Generate programs' annotation files

In [30]:
annot_utils= annotation_utilities()

In [31]:
annot_utils.generate_json_annotations(str(cluster_utils.data_path))
# annot_utils.generate_json_annotations_parallel(str(data_path),nb_threads=8)

100%|██████████| 12/12 [00:03<00:00,  3.44it/s]


## 6. Save execution results into a dataset file

In [32]:
dataset_utils = dataset_utilities()
dataset_utils.save_pkl_dataset(str(cluster_utils.data_path), 
                 cluster_utils.tmp_files_dir+'results_'+cluster_utils.batchName+'/parts/',
                 result_dataset_file)


  0%|          | 0/12 [00:00<?, ?it/s]

nb schedules 759


100%|██████████| 12/12 [00:00<00:00, 32.75it/s]


## 7. Test the resulting dataset (optional)

In [33]:
df = dataset_utils.get_dataset_df(result_dataset_file)
df

100%|██████████| 12/12 [00:00<00:00, 5009.62it/s]

Dataset location:  ./time_measurement/processed_datasets/dataset_batch_demo.pkl
Number of schedules 395
Schedules that didn't run: 0.25%
Speedups >1 :27.59%
Speedups >2 :7.85%
Speedups <0.1 :4.56%
Speedups 0.9<s<1.1 :19.75%
Mean speedup: 1.17
Median speedup: 0.81
Max speedup: 12.91
Min speedup: 0.022
Speedup variance : 3.303
Mean execution time: 0.509s
Max execution time: 15.004s


,function,schedule,exec_time,speedup
0,function002020,function002020_no_schedule,36.7585,1.000000
1,function002020,function002020_schedule_0000,39.5655,0.929054
2,function002020,function002020_schedule_0001,24.4940,1.500714
3,function002020,function002020_schedule_0002,354.2870,0.103753
4,function002020,function002020_schedule_0003,57.8945,0.634922
...,...,...,...,...
390,function002034,function002034_schedule_0027,38.2720,0.583325
391,function002034,function002034_schedule_0028,32.4420,0.688151
392,function002034,function002034_schedule_0029,45.3650,0.492119
393,function002034,function002034_schedule_0030,38.0245,0.587121
